In [5]:
import numpy as np

In [6]:
from math import sqrt

In [54]:
class GSM:
    def __init__(self, ndims, nscales):
        self.mu = np.zeros((ndims, 1))
        self.precision = np.eye(ndims)
        self.scales = list(range(1, nscales+1))
        self.weights = np.ones((nscales, 1)) / nscales

        self.nscales = nscales
        self.ndims = ndims
        
    def eval(self, x):
        
        ndims   = self.ndims;
        nscales = self.nscales;
        ndata   = x.shape[0] # or maybe 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
        
        exponent = np.exp(-0.5 * self.scales * maha)
        p = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        
        p = np.sum(p, axis=0)
        
        return p
    
    def log_grad_x(self, x):
        ndims   = self.ndims;
        nscales = self.nscales;
        ndata   = x.shape[0] # or maybe 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
            P_x_mu = self.precision[j] * x_mu
        
        exponent = np.exp(-0.5 * self.scales * maha)
        y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        y = y / np.sum(y, axis=0) # or maybe 1
        
        g = np.zeros([ndims, ndata])
        
        for s in range(1, nscales):
            g = g - self.scales[s] * P_x_mu * y
            
        return g

    def log_grad_weights(self, x):
        ndims   = self.ndims;
        nscales = self.nscales;
        ndata   = x.shape[0] # or maybe 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
        
        exponent = np.exp(-0.5 * self.scales * maha)
        y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        
        invld_arr = np.sum(y, axis = 0)
        invld = np.where(invld_arr <=0)
        y[:invld] = []
        
        gamma = y / np.sum(self.weights * y ,axis=0)
        g = np.sum(gamma, axis=1) - ndata
        g = np.reshape(g, self.weights.shape)
        
        return g
    
    def sample(self, nsamples):
        ndims = len(self.mean)
        nscales = len(self.weights)
        self.weights = self.weights[::-1]
        
        cw = np.cumsum(self.weights, axis=0)
        
        mat_1 = np.tile(np.random.rand(1, nsamples), (nscales, 1))
        mat_2 = np.tile(cw, (1, nsamples))
        mix_comp = np.sum(mat_1 <= mat_2, axis=0)
  
        # Whitening transform of precision
        if len(self.precision.shape) == 3:
            tmp = np.zeros((ndims, nsamples))
            r = np.random.randn(ndims, nsamples)

            for j in range(1, nscales+1):
                tmp2 = np.linalg.lstsq(np.linalg.cholesky(self.precision[j]), r)
                idx = np.tile(mix_comp == j, (ndims, 1))
                tmp[idx] = tmp2[idx]
        else:
            r = np.random.randn(ndims, nsamples)
            tmp = np.linalg.lstsq(np.linalg.cholesky(self.precision), r)
  
        
        # To-DO: Fix this line
        mat_3 = np.tile(np.sqrt(self.scales[mix_comp]), (ndims, 1))
        y = np.tile(self.mean, (1, nsamples)) + np.divide(tmp, mat_3)
        
        return y
    
    
    def z_distribution(self, x):
        ndims = self.ndims
        nscales = self.nscales
        ndata = x.shape[0] # Or maybe index 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
            
        exponent = np.exp(-0.5 * self.scales * maha)
        y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        
        p = y / np.sum(y, axis=0)
        
        return p
    
    
    def em(self, x, niters):
        ndims = self.ndims
        nscales = self.nscales
        ndata = x.shape[0] # Or maybe index 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
            
        for i in range(1, niters+1):
            exponent = np.exp(-0.5 * self.scales * maha)
            y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
            
        invld_arr = np.sum(y, axis = 0)
        invld = np.where(invld_arr <=0)
        y[:invld] = 1
        
        gamma = np.divide(y, np.tile(np.sum(y, axis=0), (nscales, 1)))
        self.weights = np.mean(gamma, axis=1)

        

Test GSM Class

In [55]:
R = np.sqrt(2) * np.array([[1, 1], [-1, 1]])
d = GSM(2, 4);
d.mean = np.array([[0], [1]])
d.precision = R * np.diag([0.5, 1]) * np.linalg.inv(R)
# d.scales = np.logspace(0.1, 2, 4)\

  

In [56]:
x = d.sample(1)

[3]
[1, 2, 3, 4]


<ipython-input-54-660cd2bfa02e>:108: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  tmp = np.linalg.lstsq(np.linalg.cholesky(self.precision), r)


TypeError: only integer scalar arrays can be converted to a scalar index